In [ ]:
from __future__ import annotations

import json
from pathlib import Path
from typing import Any, List

import pandas as pd

RESULTS_PATH = Path(__file__).resolve().parent / "results.jsonl"

def load_results(path: Path) -> List[dict[str, Any]]:
    if not path.exists():
        return []
    records: List[dict[str, Any]] = []
    with path.open("r", encoding="utf-8") as handle:
        for line in handle:
            line = line.strip()
            if not line:
                continue
            try:
                records.append(json.loads(line))
            except json.JSONDecodeError:
                continue
    return records

records = load_results(RESULTS_PATH)

if not records:
    records = [
        {"step": idx, "metrics": {"energy": 0.25 + idx * 0.05, "throughput": 18 + idx * 1.5, "fairness": 0.8 + idx * 0.02, "latency": 12 - idx * 0.4, "handoff_success": 0.6 + idx * 0.05}}
        for idx in range(6)
    ]

rows: List[dict[str, Any]] = []
for entry in records:
    row = {"step": entry.get("step")}
    row.update(entry.get("metrics", {}))
    rows.append(row)

df = pd.DataFrame(rows).set_index("step").sort_index()
df.head()

In [ ]:
import matplotlib.pyplot as plt

plt.style.use("seaborn-v0_8")
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
df[["energy", "throughput"]].plot(ax=axes[0], marker="o", title="Energy / Throughput")
axes[0].set_ylabel("Value")
df[["fairness", "handoff_success"]].plot(ax=axes[1], marker="o", title="Fairness / Handoff")
axes[1].set_ylabel("Ratio")
plt.tight_layout()
plt.show()